<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/jcc/00-bow%2Bbinario%2Bsvm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline JCC: BoW+SVM

En esta notebook se presetan los experimentos sobre la estrategia de representación y técnica de aprendizaje *baseline* utilizada para las JCC de  la Universidad Nacional de La Plata.

Para ello vamos a preprocesar los correos y aplicar:
- Bag of words,
- Pesado binario/no binario,
- Máquina de vector soporte (SVM).


In [1]:
# Cargamos el archivo con las consultas para entrenamiento que está en Github
from os import path

# En caso que no esté el archivo en Colab lo traigo
if not(path.exists('correos-train-80.csv')):
  !wget https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/consolidado_jcc/correos-train-80.csv  

# Leemos el archivo en un dataframe
import pandas as pd
df_train = pd.read_csv('correos-train-80.csv')

--2021-03-19 10:56:47--  https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/consolidado_jcc/correos-train-80.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223169 (218K) [text/plain]
Saving to: ‘correos-train-80.csv’

correos-train-80.cs 100%[===================>] 217.94K  --.-KB/s    in 0.03s   

2021-03-19 10:56:47 (6.77 MB/s) - ‘correos-train-80.csv’ saved [223169/223169]



In [2]:
# Me traigo las etiquetas de las clases de train_test_data.ipynb
import numpy as np

etiquetas_clases = np.array(['Boleto Universitario', 
                    'Cambio de Carrera', 
                    'Cambio de Comisión',
                    'Carga de Notas', 
                    'Certificados Web', 
                    'Consulta por Equivalencias',
                    'Consulta por Legajo', 
                    'Consulta sobre Título Universitario',
                    'Cursadas', 
                    'Datos Personales', 
                    'Exámenes',
                    'Ingreso a la Universidad',
                    'Inscripción a Cursadas',
                    'Pedido de Certificados',
                    'Problemas con la Clave',
                    'Reincorporación', 
                    'Requisitos de Ingreso',
                    'Simultaneidad de Carreras', 
                    'Situación Académica',
                    'Vacunas Enfermería'])

In [3]:
# Transformamos todas las Clases minoritarias (Puedo ir variando la cantidad de clases que derivo a la Clase "Otras Consultas")
cantidad_clases=4

clases = df_train.clase.value_counts()
clases_minoritarias = clases.iloc[cantidad_clases-1:].keys().to_list()
clases_minoritarias

# Agrego a las etiquetas la etiqueta "Otras Consultas"
etiquetas_clases = np.append(etiquetas_clases, "Otras Consultas")

# Genero una nueva clave de clases para "Otras Consultas" a modo de agrupar las que poseen menos apariciones
df_train.clase[df_train['clase'].isin(clases_minoritarias)] = np.where(etiquetas_clases == "Otras Consultas")[0]


# Clases balanceadas
df_train.clase.value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


20    320
0     192
11    185
16    103
Name: clase, dtype: int64

In [4]:
# Me guardo los atributos, excepto la clase en x
y_train = df_train['clase'].to_numpy()
x_train = df_train.drop(['clase'], axis=1)

## SVM

Generamos el modelo:

In [5]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC

# Defino la feature y la transformación a aplicar en el texto
text_features = 'Consulta'
text_transformer = CountVectorizer()

# Defino la transformación
preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_transformer, text_features)
        ], remainder='passthrough')

# Combino la transformación con el pipeline
model_pipe = Pipeline([('preprocessor', preprocessor),
                       ('svm', SVC())])

# Defino los parámetros para GridSearchCV
parameters=[
        {'preprocessor__text__binary': [True, False],
         'preprocessor__text__analyzer': ['char'],
         'preprocessor__text__ngram_range': ((3, 3), (4, 7), (3, 4)),
#         'preprocessor__text__strip_accents': ['unicode'],     
#         'preprocessor__text__max_features':[500, 1000, 1500, 3000],
#         'svm__C': [0.1, 1, 10, 100, 1000],  
         'svm__gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
#         'svm__class_weight': [None, 'balanced'],
#         'svm__kernel': ['rbf', 'linear', 'poly', 'rbf', 'sigmoid']
    }
]

# Instancio y "entreno" el GridSearchCV
grid_search=GridSearchCV(model_pipe, param_grid=parameters, cv=None, n_jobs=-1, verbose=3)
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   34.6s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  4.0min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('text',
                                                                         CountVectorizer(analyzer='word',
                                                                                         binary=False,
                                                                                         decode_error='strict',
                                                                                         dtype=<class 'numpy.int64'>,
                      

In [6]:
# Imprimo los mejores parámetros encontrados por GridSearchCV
print(grid_search.best_params_) 
  
# Imprimo el modelo después del ajuste de hiperparámetros
print(grid_search.best_estimator_) 

{'preprocessor__text__analyzer': 'char', 'preprocessor__text__binary': False, 'preprocessor__text__ngram_range': (4, 7), 'svm__gamma': 0.001}
Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('text',
                                                  CountVectorizer(analyzer='char',
                                                                  binary=False,
                                                                  decode_error='strict',
                                                                  dtype=<class 'numpy.int64'>,
                                                                  encoding='utf-8',
                                                                  input='content',
                                        

In [7]:
# Me autentico en Drive
from google.colab import drive
drive.mount('drive')

# Paso los resultados y el accuracy a un dataframe
results = pd.concat([pd.DataFrame(grid_search.cv_results_["params"]), pd.DataFrame(grid_search.cv_results_["mean_test_score"], columns=["Accuracy"])],axis=1)

# Los guardo en data_results.csv en mi Drive
results.to_csv('data_results.csv')
!cp data_results.csv "drive/My Drive/"

Mounted at drive


In [8]:
# Levanto las instancias de testeo

# En caso que no esté el archivo en Colab lo traigo
if not(path.exists('correos-test-20.csv')):
  !wget https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/consolidado_jcc/correos-test-20.csv  

# Leemos el archivo en un dataframe
import pandas as pd
df_test = pd.read_csv('correos-test-20.csv')

--2021-03-19 11:01:49--  https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/consolidado_jcc/correos-test-20.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58146 (57K) [text/plain]
Saving to: ‘correos-test-20.csv’

correos-test-20.csv 100%[===================>]  56.78K  --.-KB/s    in 0.01s   

2021-03-19 11:01:49 (4.31 MB/s) - ‘correos-test-20.csv’ saved [58146/58146]



In [9]:
# Balanceo las clases de testing

clases_test = df_test.clase.value_counts()
clases_minoritarias_test = clases_test.iloc[cantidad_clases-1:].keys().to_list()
clases_minoritarias_test

# Genero una nueva clave de clases para "Otras Consultas" a modo de agrupar las que poseen menos apariciones
df_test.clase[df_test['clase'].isin(clases_minoritarias)] = np.where(etiquetas_clases == "Otras Consultas")[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


20    79
0     48
11    47
16    26
Name: clase, dtype: int64

In [10]:
# Me guardo los atributos, excepto la clase en x
y_test = df_test['clase'].to_numpy()
x_test = df_test.drop(['clase'], axis=1)

In [11]:
from sklearn.metrics import classification_report, confusion_matrix 

# Se realizan las predicciones sobre el conjunto de validación
grid_predictions = grid_search.predict(x_test) 

# Se imprime el reporte de clasificación
print(classification_report(y_test, grid_predictions))

              precision    recall  f1-score   support

           0       1.00      0.67      0.80        48
          11       0.76      0.34      0.47        47
          16       1.00      0.15      0.27        26
          20       0.52      0.94      0.67        79

    accuracy                           0.63       200
   macro avg       0.82      0.52      0.55       200
weighted avg       0.75      0.63      0.60       200



# Referencias
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
- https://medium.com/analytics-vidhya/ml-pipelines-using-scikit-learn-and-gridsearchcv-fe605a7f9e05